In [1]:
import sys
sys.path.append("..")
from common_utils import set_data_home
set_data_home("~/datasets")
from common_utils import DATA_HOME, join
from lstm.sales_data import Sales_Dataset
from torch.utils.data.dataloader import DataLoader
import torch


SALE_HOME = join(DATA_HOME, "sales_data")
MODEL_HOME = join(DATA_HOME, "sale_model")


In [2]:
from torch.nn import LSTM

I, H, B = 71, 34, 3

sd = Sales_Dataset(SALE_HOME, seq_len=500, is_train=False)

model = LSTM(I, H, batch_first=True).cuda()
model.load_state_dict(torch.load("./old_sales_model_1000.pth"))
sd[0]
sd[1]
sd[16]
sd[17]

(tensor([[-0.9542,  0.0000,  0.0000,  ..., -1.6550,  0.8165, -1.4142],
         [ 0.6990,  0.0000,  0.0000,  ..., -1.6550,  0.8165, -1.4142],
         [-0.2218,  0.0000,  0.0000,  ..., -1.6550,  0.8165, -1.4142],
         ...,
         [-0.3522,  0.0000,  0.0000,  ..., -1.6550,  0.8165, -1.4142],
         [-0.3010,  0.0000,  0.0000,  ..., -1.6550,  0.8165, -1.4142],
         [ 0.0000,  0.0000,  0.0000,  ..., -1.6550,  0.8165, -1.4142]],
        device='cuda:0'),
 tensor([[ 0.6990,  0.0000,  0.1249,  ...,  1.0000, -0.1069,  0.0204],
         [-0.2218,  0.0000,  0.2430,  ...,  0.0000,  0.0148, -0.0106],
         [ 0.0000,  0.0000, -0.1461,  ..., -0.1761, -0.0964, -0.0190],
         ...,
         [-0.3010,  0.0000,  0.1091,  ...,  0.0000, -0.0248, -0.0126],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
 2,
 Timestamp('2017-08-17 00:00:00'))

In [3]:
import pandas as pd
ans_df = pd.DataFrame()
for X, y, store_id, date in sd:
    yhat, (_, _) = model(X.cuda())
    # sd.set_log_rets(yhat[-1, :], store_id, date)
    print(f"store_id: {store_id}, date: {date} infered")

sd.TR.to_csv("./TR.csv", index=False)

store_id: 1, date: 2017-08-16 00:00:00 infered
store_id: 1, date: 2017-08-17 00:00:00 infered
store_id: 1, date: 2017-08-18 00:00:00 infered
store_id: 1, date: 2017-08-19 00:00:00 infered
store_id: 1, date: 2017-08-20 00:00:00 infered
store_id: 1, date: 2017-08-21 00:00:00 infered
store_id: 1, date: 2017-08-22 00:00:00 infered
store_id: 1, date: 2017-08-23 00:00:00 infered
store_id: 1, date: 2017-08-24 00:00:00 infered
store_id: 1, date: 2017-08-25 00:00:00 infered
store_id: 1, date: 2017-08-26 00:00:00 infered
store_id: 1, date: 2017-08-27 00:00:00 infered
store_id: 1, date: 2017-08-28 00:00:00 infered
store_id: 1, date: 2017-08-29 00:00:00 infered
store_id: 1, date: 2017-08-30 00:00:00 infered
store_id: 1, date: 2017-08-31 00:00:00 infered
store_id: 2, date: 2017-08-16 00:00:00 infered
store_id: 2, date: 2017-08-17 00:00:00 infered
store_id: 2, date: 2017-08-18 00:00:00 infered
store_id: 2, date: 2017-08-19 00:00:00 infered
store_id: 2, date: 2017-08-20 00:00:00 infered
store_id: 2, 

### Transform returns to sales

In [4]:
from datetime import timedelta
import pandas as pd
sales = pd.read_csv(join(SALE_HOME, "test.csv"), index_col=False)
sales["sales"] = 0.
sales.sort_values(["store_nbr", "family", "date"], inplace=True)
# sd.TR = pd.read_csv(join(SALE_HOME, "TR.csv"), index_col=False)
sd.TR = sd.TR.reset_index().sort_values(["store_nbr", "family", "date"])
base_sales = sd.base_sales

for s in sd.ids:
    for f in sd.families:
        start_date, end_date = sd.train_max_date + timedelta(days=1), sd.test_max_date
        prev_sale = base_sales.loc[(base_sales.store_nbr == s) & (base_sales.family == f)].sales
        for d in pd.date_range(start_date, end_date):
            ret = sd.TR.loc[(sd.TR.store_nbr == s) &\
                            (sd.TR.family == f) & \
                            (sd.TR.date == d)].sales
            curr_sale = 10 ** ret.item() * prev_sale.item()
            sales.loc[(sales.store_nbr == s) & (sales.family == f) & (sales.date == d.strftime("%Y-%m-%d")), "sales"] = curr_sale
        print(f"store {s}, family {f} inferred")
            

base_sales, sd.test_max_date - sd.train_max_date, sales

store 1, family AUTOMOTIVE inferred
store 1, family BABY CARE inferred
store 1, family BEAUTY inferred
store 1, family BEVERAGES inferred
store 1, family BOOKS inferred
store 1, family BREAD/BAKERY inferred
store 1, family CELEBRATION inferred
store 1, family CLEANING inferred
store 1, family DAIRY inferred
store 1, family DELI inferred
store 1, family EGGS inferred
store 1, family FROZEN FOODS inferred
store 1, family GROCERY I inferred
store 1, family GROCERY II inferred
store 1, family HARDWARE inferred
store 1, family HOME AND KITCHEN I inferred
store 1, family HOME AND KITCHEN II inferred
store 1, family HOME APPLIANCES inferred
store 1, family HOME CARE inferred
store 1, family LADIESWEAR inferred
store 1, family LAWN AND GARDEN inferred
store 1, family LINGERIE inferred
store 1, family LIQUOR,WINE,BEER inferred
store 1, family MAGAZINES inferred
store 1, family MEATS inferred
store 1, family PERSONAL CARE inferred
store 1, family PET SUPPLIES inferred
store 1, family PLAYERS AND

(      store_nbr       date                      family     sales  onpromotion
 0             1 2017-08-15                  AUTOMOTIVE     4.000            0
 1             1 2017-08-15                   BABY CARE     0.000            0
 2             1 2017-08-15                      BEAUTY     4.000            1
 3             1 2017-08-15                   BEVERAGES  1942.000           11
 4             1 2017-08-15                       BOOKS     0.000            0
 ...         ...        ...                         ...       ...          ...
 1777         54 2017-08-15                     POULTRY    59.619            0
 1778         54 2017-08-15              PREPARED FOODS    94.000            0
 1779         54 2017-08-15                     PRODUCE   915.371           76
 1780         54 2017-08-15  SCHOOL AND OFFICE SUPPLIES     0.000            0
 1781         54 2017-08-15                     SEAFOOD     3.000            0
 
 [1782 rows x 5 columns],
 Timedelta('16 days 00:0

### output the answer

In [5]:
sales.drop(columns=["store_nbr", "date", "family", "onpromotion"]).sort_values("id").to_csv("answer.csv", index=False)